In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
import numpy as np
import matplotlib.pyplot as plt 
import pandas as pd
import sklearn
import keras
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import *
from tensorflow.keras.applications import *
from keras.preprocessing.image import ImageDataGenerator
import seaborn as sns               
import glob, warnings
from sklearn.metrics import confusion_matrix, classification_report
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
traindf=pd.read_csv('/content/drive/MyDrive/Deepl learning hackathon/train_(4).csv', dtype='str')
traindf.head()

In [ ]:
traindf['file_id'] = traindf['file_id'] + '.jpg'
traindf.head()

In [ ]:
count = traindf['label'].value_counts() 
print(count) 

In [ ]:
train_path = '/content/drive/MyDrive/Deepl learning hackathon/all'
test_path = '/content/drive/MyDrive/Deepl learning hackathon/test_(3).csv'

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(traindf, test_size = 0.1, random_state = 42)
print(train.shape)
print(test.shape)

In [ ]:
datagen_train = ImageDataGenerator(
    rescale=1/244)


datagen_val = ImageDataGenerator(
      rescale=1/244)

In [ ]:
train_set = datagen_train.flow_from_dataframe(
    train,
    directory=train_path,
    seed=42,
    x_col='file_id',
    y_col="label",
    target_size = (300,300),
    class_mode='categorical',
    color_mode = 'rgb',
    shuffle = True,
    batch_size = 32,
)
test_set = datagen_val.flow_from_dataframe(
    test,
    directory=train_path,
    seed= 42,
    x_col='file_id',
    y_col="label",
    target_size = (300,300),
    color_mode = 'rgb',
    class_mode='categorical',
    shuffle=False,
    batch_size= 32,    
)



In [ ]:
IMG_SIZE = 300
n_CLASS = 2

pretrained_model =  tf.keras.applications.inception_v3.InceptionV3(
            input_shape = (IMG_SIZE, IMG_SIZE, 3), 
            include_top = False,
            weights='imagenet',
        )
def create_model():
    
    model = Sequential()
    # initialize the model with input shape
    model.add(pretrained_model)
    model.add(Flatten())
    model.add(Dense(
        300, 
        activation= 'relu'
    ))
    model.add(Dense(30, activation = 'relu'))
    model.add(Dropout(0.4))
    model.add(Dense(n_CLASS, activation = 'softmax'))
    
    return model
fake_model = create_model()

In [ ]:
fake_model.summary()

In [ ]:
tf.keras.utils.plot_model(fake_model)

In [ ]:
learning_rate = 1e-3
optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate)

fake_model.compile(optimizer = optimizer, 
              loss = tf.keras.losses.CategoricalCrossentropy(), 
              metrics = 'accuracy')

In [ ]:
int_epoch = 15
history = fake_model.fit(x = train_set,
          validation_data = test_set,
          epochs = int_epoch)    

In [ ]:
def load_trained_model(weights_path):
   model = create_model()
   model.load_weights(weights_path)
   return model

In [ ]:
new_model = load_trained_model('./fake1.hdf5')
new_model.summary()

In [ ]:
main_test_path = '/content/drive/MyDrive/Deepl learning hackathon/test_(3).csv'
main_test_images = '/content/drive/MyDrive/Deepl learning hackathon/test-20211014T110228Z-001'

In [ ]:
test_img=pd.read_csv(main_test_path,dtype='str')
test_img['file_id']=test_img['file_id']+'.jpg'
test_img.head(2)

In [ ]:
aa='/content/drive/MyDrive/Deepl learning hackathon/test-20211014T110228Z-001/test'

In [ ]:
main_test = datagen_val.flow_from_dataframe(
    test_img,
    directory=aa,
    seed= 42,
    x_col='file_id',
    y_col=None,
    target_size = (300,300),
    color_mode = 'rgb',
    class_mode=None,
    shuffle=False,
    batch_size= 32,    
)


In [ ]:
y_pred = new_model.predict(main_test)

In [ ]:
y_predargmax = tf.argmax(y_pred , 1)
print(y_predargmax)

In [ ]:
sub=pd.read_csv(main_test_path,dtype='str')
sub['label']=y_predargmax
sub.to_csv('result3.csv',index=False)